In [63]:
import numpy as np

# Project 2: Numerical Quantum Mechanics

## What files should I submit?

* proj_2_module.py
* proj_2_EigenTest.ipynb
* proj_2_EigenTest.pdf
* proj_2_EigenAnalysis.ipynb
* proj_2_EigenAnalysis.pdf

The .pdf are converted from .ipynb.

## Write a Python module to compute eigenvalues and eigenvectors

The module should contain a function

    hermitian_eigensystem(H,tolerance)

allowing you to numerically determine the eigenvalues and eigenvectors of a hermitian matrix of any size.  

```H``` is a hermitian square matrix.

```tolerance``` is a small number, such that we consider $M$ is diagonal when:

$$
\text{off}[M] \leq \text{tolerance} * \text{norm}[M]
$$

($\text{off}[M]$ and $|M|$ defined in the lecture notes.)

The module can contain any other functions you wish that might help you divide and conquer the diagonalization task into relevant sub-parts.  The eigenvalues should be output in non-decreasing order, and the corresponding eigenvectors should be listed in corresponding order.  The module's name should be:

    proj_2_module.py


You may use ```numpy.linalg.eig``` or ```scipy.linalg.eig``` to check your method, but do not put them secretly in your module.

### hint
It is highly recommended that you split your functions into many parts.
You may design your own strategy. Below is one example.

* (A) jacobi_rotation
* (O) off
* (B) real_eigen calls (A) iteratively,  (B) also calls (O) to check if the matrix is consider as diagonal
* (C) complex_eigen calls (B)
* (D) hermitian_eigensystem calls (C), and makes the output ordered.

In [91]:
#difficulty: ★★★
def jacobi_rotation(A, j, k):
    #Args:
        # A (np.ndarray): n by n real symmetric matrix
        # j (int): column parameter.
        # k (int): row parameter.

    #Returns:
        # A (np.ndarray): n by n real symmetric matrix, where the A[j,k] and A[k,j] element is zero
        # J (np.ndarray): n by n orthogonal matrix, the jacobi_rotation matrix

    # getting the element for A
    a = A[j, j]
    b = A[k, k]
    c = A[j, k]
    d = A[k, j]

    theta = np.arctan2(2*c, a-b)
    
    J = np.diag(np.full(A.shape[0], 1))
    J[j, j] = np.cos(theta)
    J[j, k] = np.sin(theta)
    J[k, j] = -np.sin(theta)
    J[k, k] = np.cos(theta)
    A = J.transpose() @ A @ J
    A[j, k] = 0
    A[k, j] = 0
    
    return A, J

#difficulty: ★
def off(A, tol = 1e-5):
    # see lecture note equation (12) 
    A_off = np.abs(A - np.diag(np.diag(A)))
    output = np.all(A_off < tol)
    return output

def find_largest_non_diagonal_index(matrix):
    # Ensure the matrix is a square matrix
    assert matrix.shape[0] == matrix.shape[1], "The matrix must be square."
    
    # Determine the minimum value based on the data type of the elements in the matrix
    if np.issubdtype(matrix.dtype, np.integer):
        min_value = np.iinfo(matrix.dtype).min
    elif np.issubdtype(matrix.dtype, np.floating):
        min_value = -np.finfo(matrix.dtype).max
    else:
        raise TypeError("Unsupported data type for matrix elements")
    
    # Create a copy of the matrix to avoid modifying the original
    temp_matrix = matrix.copy()
    
    # Set the diagonal elements to the determined minimum value
    np.fill_diagonal(temp_matrix, min_value)
    
    # Find the index of the largest element in the modified matrix
    # np.argmax returns the index in flattened array, so use np.unravel_index to convert it to a tuple of (row, column)
    index = np.unravel_index(np.argmax(temp_matrix), matrix.shape)
    
    return index

#difficulty: ★★★
def real_eigen(A,tolerance):
    #Args:
        # A (np.ndarray): n by n real symmetric matrix
        # tolerance (double): the relative precision
    #Returns:
        # d (np.ndarray): n by 1 vector, d[i] is the i-th eigenvalue
        # R (np.ndarray): n by n orthogonal matrix, R[:,i] is the i-th eigenvector
        
    # call jacobi_rotation(A, j, k) iteratively
    # call off and norm to check if we can stop the iteration
    # off (you write it)
    # norm https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.norm.html
    
    stop = False
    J_arr = []

    while (not stop):
        j, k = find_largest_non_diagonal_index(A)
        A, J = jacobi_rotation(A, j, k)
        J_arr.append(J)

        # calculate the Frobenius norm of A and off(A)
        # to see if we want to stop the call
        A_off = A.copy()
        np.fill_diagonal(A_off, 0)
        off_norm = np.linalg.norm(A_off, 'fro')
        frob_norm = np.linalg.norm(A, 'fro')
        stop = not ((off_norm/frob_norm < tolerance) and (off(A)))

    R = np.diag(np.full(A.shape[0], 1))
    d = np.diagonal(A)
    for J_i in J_arr:
        R = R*J_i
    return d, R

In [92]:
import qutip as qt

x = (qt.basis(2, 0) + qt.basis(2, 1)).unit()
psi = qt.tensor(x, qt.basis(2, 0))
psi = x
rho = psi * psi.dag()
eigenstates,eigenvectors = rho.eigenstates()
for vec in eigenvectors:
    print(vec)
print(eigenstates)
rho = np.abs(rho.full())

Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[-0.70710678]
 [ 0.70710678]]
Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.70710678]
 [0.70710678]]
[0. 1.]


In [93]:
d, R = real_eigen(rho, 1e-6)
print(d)
print(R)

KeyboardInterrupt: 

In [75]:
# #difficulty: ★★
# def complex_eigen(H,tolerance):
#     #Args:
#         # A (np.ndarray): n by n real hermitian matrix
#         # tolerance (double): the relative precision
#     #Returns:
#         # d (np.ndarray): n by 1 vector, d[i] is the i-th eigenvalue
#         # U (np.ndarray): n by n unitary matrix, U[i,:] is the i-th eigenvector
        
#     #(1) split H=S+iA to real and imaginary parts
#     #(2) construct a 2n by 2n real matrix B
#     #(3) call dd, R = real_eigen(B,tolerance)
#     #(4) organize the output of dd, R  to get d and U
        
#     return d, U

In [68]:
# #difficulty: ★
# def hermitian_eigensystem(H,tolerance):
    
#     # call complex_eigen(H,tolerance)
#     # rearrange d and U, so that they are in the non-decreasing order of eigenvalues
#     return d, U

## Write a Jupyter notebook containing code tests

The notebook is meant to validate the performance of your eigensystem module and should be named as:

    proj_2_EigenTest.ipynb


The notebook should contain:

Tests showing that for hermitian matrices of sizes up to 30-by-30 with known eigenvalues, the function `hermitian eigensystem` gives correct eigenvalues and eigenvectors.  To generate test cases, you'll need to think about how you can generate hermitian matrices with known eigenvalues.  Hint: what happens when you apply a similarity transformation by a unitary matrix to a diagonal matrix?  You may also find it useful to look into the function 'scipy.stats.unitary_group' which allows one to generate random unitary matrices.

## Write a Jupyter notebook analyzing the anharmonic oscillator

This notebook should use your eigensystem module to determine the first few eigenvalues and corresponding eigenvectors of the anharmonic oscillator hamiltonian.  The notebook should be named as:

    proj_2_EigenAnalysis.ipynb
        
1. Show that the operators $\hat x^2$ and $\hat x^4$ have the following matrix elements in the harmonic oscillator basis:
    \begin{align}
    \langle n|\hat x^2|m\rangle 
    &=(n+1/2)\delta_{nm} + \tfrac{1}{2}\sqrt{(n+1)(n+2)}\,\delta_{n,m-2} + 
\tfrac{1}{2}\sqrt{(n-1)n\,}\,\delta_{n,m+2} \\
    \langle n|\hat x^4|m\rangle 
    &= \tfrac{1}{4}\!\left(6n^2 + 6n + 3\right)\!\delta_{nm}
+ \sqrt{(n+1)(n+2)}\left(n+\tfrac{3}{2}\right)\!\delta_{n,m-2}\;+ \nonumber\\
& + \sqrt{(n-1)n\,}\left(n-\tfrac{1}{2}\right)\!\delta_{n,m+2} +
\tfrac{1}{4}\sqrt{(n+1)(n+2)(n+3)(n+4)}\,\delta_{n,m-4}\;+ \nonumber\\
& + \tfrac{1}{4}\sqrt{(n-3)(n-2)(n-1)n\,}\,\delta_{n,m+4}.
\end{align}
1. Solve the anharmonic oscillator eigenvalue problem written in the harmonic oscillator basis for at least the first four energy levels. Note that the function `hermval` from NumPy offers an easy solution to compute the eigenfunctions $\psi_n(x)$ from the eigenvectors of the matrix representation of the hamiltonian. 
1. Plot the first four energy 
levels $E_n(\lambda)$ versus $\lambda$ over the range 
$0 \leq \lambda \leq 1$. Plot also the spacings between the 
levels $\Delta E(\lambda) = E_{n+1}(\lambda) - E_n(\lambda)$. Make sure to use a basis 
size $N$ sufficiently larger than the desired number of lowest eigenvalues to ensure convergence of the eigensystem algorithm.
1. Check the convergence of the method with respect to the basis size $N$ by plotting one 
of the lowest (or more) energy eigenvalues $E_n(N)$ for $\lambda = 1$ versus the basis size $N$. 
Alternatively, to demonstrate the convergence more clearly, you can also plot the differences between 
two consecutive estimates $\epsilon_n = E_n(N) - E_n(N\!+\!2)$ versus $N$.
1. Plot and compare the first four eigenfunctions $\psi_n(x)$ for the harmonic oscillator with $\lambda=0$ to 
the eigenfunctions for the anharmonic oscillator with $\lambda=1$.